In [5]:
import cv2
import numpy as np
def cv_show(name, image):
 cv2.imshow(name, image)
 
 cv2.waitKey(0)
 cv2.destroyAllWindows()
 
def detectAndDescribe(image):
    # 将彩色图片转换成灰度图
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # 建立SIFT生成器
    descriptor = cv2.SIFT_create()
    # 检测SIFT特征点，并计算描述子
    (kps, features) = descriptor.detectAndCompute(image, None)
    # 将结果转换成NumPy数组
    kps = np.float32([kp.pt for kp in kps])
    # 返回特征点集，及对应的描述特征
    return (kps, features)
def matchKeypoints(kpsA, kpsB, featuresA, featuresB, ratio = 0.75, reprojThresh = 4.0):
    # 建立暴力匹配器
    matcher = cv2.BFMatcher()
    # 使用KNN检测来自A、B图的SIFT特征匹配对，K=2
    rawMatches = matcher.knnMatch(featuresA, featuresB, 2)
    matches = []
    for m in rawMatches:
        # 当最近距离跟次近距离的比值小于ratio值时，保留此匹配对
        if len(m) == 2 and m[0].distance < m[1].distance * ratio:
        # 存储两个点在featuresA, featuresB中的索引值
            matches.append((m[0].trainIdx, m[0].queryIdx))
        # 当筛选后的匹配对大于4时，计算视角变换矩阵
    if len(matches) > 4:
        # 获取匹配对的点坐标
        ptsA = np.float32([kpsA[i] for (_, i) in matches])
        ptsB = np.float32([kpsB[i] for (i, _) in matches])
        # 计算视角变换矩阵
        (H, status) = cv2.findHomography(ptsA, ptsB, cv2.RANSAC, reprojThresh)
        # 返回结果
        return (matches, H, status)
    # 如果匹配对小于4时，返回None
    return None
def drawMatches(imageA, imageB, kpsA, kpsB, matches, status):
    # 初始化可视化图片，将A、B图左右连接到一起
    (hA, wA) = imageA.shape[:2]
    (hB, wB) = imageB.shape[:2]
    vis = np.zeros((max(hA, hB), wA + wB, 3), dtype="uint8")
    vis[0:hA, 0:wA] = imageA
    vis[0:hB, wA:] = imageB
    # 联合遍历，画出匹配对
    for ((trainIdx, queryIdx), s) in zip(matches, status):
        # 当点对匹配成功时，画到可视化图上
        if s == 1:
            # 画出匹配对
            ptA = (int(kpsA[queryIdx][0]), int(kpsA[queryIdx][1]))
            ptB = (int(kpsB[trainIdx][0]) + wA, int(kpsB[trainIdx][1]))
            cv2.line(vis, ptA, ptB, (0, 255, 0), 1)
    cv_show("drawImg", vis)
    cv2.imwrite("drawImg.jpg",vis)
    # 返回可视化结果
    return vis
def stitch(imageA,imageB, ratio=0.75, reprojThresh=4.0,showMatches=False):
    #检测A、B图片的SIFT关键特征点，并计算特征描述子
    (kpsA, featuresA) = detectAndDescribe(imageA)
    (kpsB, featuresB) = detectAndDescribe(imageB)
    # 匹配两张图片的所有特征点，返回匹配结果
    M = matchKeypoints(kpsA, kpsB, featuresA, featuresB, ratio, reprojThresh)
    # 如果返回结果为空，没有匹配成功的特征点，退出算法
    if M is None:
        return None
    # 否则，提取匹配结果
    # H是3x3视角变换矩阵      
    (matches, H, status) = M
    # 将图片A进行视角变换，result是变换后图片
    result = cv2.warpPerspective(imageA, H, (imageA.shape[1] + imageB.shape[1], imageA.shape[0]))
    cv_show('result', result)
    # 将图片B传入result图片最左端
    result[0:imageB.shape[0], 0:imageB.shape[1]] = imageB
    cv_show('result', result)
    # 检测是否需要显示图片匹配
    if showMatches:
        # 生成匹配图片
        vis = drawMatches(imageA, imageB, kpsA, kpsB, matches, status)
        # 返回结果
        return (result, vis)
    # 返回匹配结果
    return result


In [2]:
# 读取图像
imageA = cv2.imread('jm2.jpg')
cv_show("imageA", imageA)
imageB = cv2.imread('jm1.jpg')
cv_show("imageB", imageB)
# 计算SIFT特征点和特征向量
(kpsA, featuresA) = detectAndDescribe(imageA)
(kpsB, featuresB) = detectAndDescribe(imageB)
# 基于最近邻和随机取样一致性得到一个单应性矩阵
(matches,H,status) = matchKeypoints(kpsA, kpsB, featuresA, featuresB)
print(H)
# 绘制匹配结果
drawMatches(imageA, imageB, kpsA, kpsB, matches, status)
# 拼接



[[ 1.03686536e+00  5.09785501e-02 -1.33883267e+03]
 [ 1.35746393e-02  1.03080574e+00 -9.98131415e+01]
 [ 1.20718841e-05  5.02807194e-06  1.00000000e+00]]


array([[[ 69,  64,  65],
        [ 68,  63,  64],
        [ 67,  62,  63],
        ...,
        [ 59,  84,  70],
        [ 60,  83,  69],
        [ 53,  76,  62]],

       [[ 68,  63,  64],
        [ 67,  62,  63],
        [ 68,  63,  64],
        ...,
        [ 57,  82,  68],
        [ 53,  76,  62],
        [ 48,  71,  57]],

       [[ 62,  58,  57],
        [ 64,  59,  60],
        [ 67,  62,  63],
        ...,
        [ 56,  80,  68],
        [ 50,  75,  61],
        [ 51,  76,  62]],

       ...,

       [[233, 233, 227],
        [234, 234, 228],
        [235, 235, 229],
        ...,
        [126, 138, 140],
        [134, 143, 147],
        [139, 148, 152]],

       [[233, 234, 225],
        [232, 233, 224],
        [233, 231, 223],
        ...,
        [116, 125, 128],
        [122, 131, 134],
        [131, 140, 143]],

       [[232, 233, 223],
        [230, 231, 221],
        [229, 227, 219],
        ...,
        [ 98, 107, 110],
        [103, 112, 115],
        [115, 124, 127]]

In [6]:
result,vis = stitch(imageA, imageB)

In [8]:
cv2.imwrite('result.jpg', result)

True